In [3]:
import torch
import os
from PIL import Image
import clip
import os.path as osp
import os, sys
import torchvision.utils as vutils
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
from lib.utils import load_model_weights,mkdir_p
from models.GALIP import NetG, CLIP_TXT_ENCODER

In [4]:
device = 'cpu' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

100%|███████████████████████████████████████| 338M/338M [00:11<00:00, 30.8MiB/s]


In [5]:
text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)
path = '../saved_models/pretrained/pre_coco.pth'
checkpoint = torch.load(path, map_location=torch.device('cpu'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

In [6]:
batch_size = 100
noise = torch.randn((1, 100)).to(device)

In [6]:
#captions = ['Line drawing illustration of a kawaii cute ghost.']

In [5]:
#captions = ['归纳汇总为以下八条，开车的朋友请留意。2013年1月1日开始执行']

In [9]:
mkdir_p('./samples')

In [4]:
mkdir_p('./real')

NameError: name 'mkdir_p' is not defined

In [7]:
import pandas as pd
import numpy as np
import shutil

In [25]:
#处理好的数
df=pd.read_csv('/home/jovyan/work/GALIP/test_rumor.csv')

In [26]:
df.dropna(subset=['0'], inplace=True)

In [27]:
df.reset_index(drop=True, inplace=True)

In [28]:
df=df[:100]

In [29]:
df.head()

,0,1
0,【郭美美申请低保，居然获准通过！】郭美美对记者说，她其实就是一个失业人员，并不是大家想象的那...,7fcc7edetw1e6btq03senj20c80gawgs
1,【看病要付钱只有二十几个国家】中国工程院院士戴勀戎：“看病就得付钱。在我们这里好像天经地义。...,8d5c03a7jw1e6r4cc7x02j20by0c40tt
2,东北证券吉林公主岭营业部今天上午被愤怒的股民点燃.,3dc2ac30jw1dx457fr1owj
3,【交通新规】归纳汇总为以下八条，开车的朋友请留意。2013年1月1日开始执行。@于建嵘 @海...,96449d00jw1dyupdyukbkj
4,隐翅虫,5103ab97gw1dwmttofzfoj


In [32]:
len(df.iloc[0][0])

71

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       100 non-null    object
 1   1       100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [33]:
def fun_x(x):
    if len(x)>20: 
        return x[:20]
    else:
        return x

In [34]:
df['0'] = df['0'].apply(fun_x)

In [58]:
captions = df['0'].tolist()

In [36]:
captions

['【郭美美申请低保，居然获准通过！】郭美美',
 '【看病要付钱只有二十几个国家】中国工程院',
 '东北证券吉林公主岭营业部今天上午被愤怒的',
 '【交通新规】归纳汇总为以下八条，开车的朋',
 '隐翅虫',
 '@aKing健: nb//@doggie',
 '#塘沽爆炸真相##天津爆炸#怎么可能就5',
 '被和谐好几次了。继续吧。还有靠谱的人联系',
 '【中国是超级大国！】清华大学院长阎学通：',
 '【请民警同志把被告人押出去！】审判长先生',
 '昨日，温州申银万国证劵一女子跳楼。股市大',
 '世界那么大，那里才安全？ 2兰州·榆中县',
 '摸奶无罪:请尽情摸MM的&quot;奶&',
 '#马航飞机失联#看完这张照片你一定会质疑',
 '蓄谋已久的复旦投毒。林某本想杀姜成，误杀',
 '看来中日真的要开打了！局势紧张啊！',
 '【内幕2 来袭！图片！】  就因为刚才那',
 '大家切記！別亂溝女 ..好恐怖好恐怖～@',
 '#东京申奥成功# 东京奥运预算为北京1/',
 '@ 八双鞋；陕西永寿刘爱玲惨死，震动了整',
 '【刘丽&amp;李玫瑾】刘丽是个普通的劳',
 '太震憾了！这是拍自四川大凉山区、贵州毕节',
 '#重庆10名官员涉不雅视频被免职 请颁赵',
 '最近又有人开始质疑张海迪的国籍以及他为什',
 '《著名演员姜文去世！撼动人心》 昨天，著',
 '【巅峰一号昨天也半空&quot;卡&qu',
 '2011深圳大运800亿，2010广州亚',
 '中国政府公信力何在？一个没有公信力的政府',
 '保护好宝贝！万达广场出现偷娃娃的情况，有',
 '**教委紧急通知**:牛肉不许吃了。未接',
 '文章和高晓松是一个学校毕业的？语文是一个',
 '@洛兵:回复@梦里依稀看今朝:不会黑暗到',
 '#房祖名吸毒被抓#警方未公布的120人名',
 '通缉令：这11个人是昨晚昆明惨案的暴徒，',
 '水淹武汉，淹出中国的一面',
 '大赦天下 【官场百态】卢氏县原县委书记杜',
 '万能的圈求扩散 朋友在三亚旅游遇到一个可',
 '信阳的汽车、房地产市场要火爆了！',
 '#天津塘沽大爆炸#&nbsp;&nbsp',
 '【河南省委宣传部邀请周小平上班，档案局干',
 '【世界上唯一拒绝特蕾莎修女的国家】特蕾莎',
 '中国人八成

In [59]:
file_name = df['1'].tolist()

In [60]:
file_name

['7fcc7edetw1e6btq03senj20c80gawgs',
 '8d5c03a7jw1e6r4cc7x02j20by0c40tt',
 '3dc2ac30jw1dx457fr1owj',
 '96449d00jw1dyupdyukbkj',
 '5103ab97gw1dwmttofzfoj',
 '3d5b0116jw1e3s876pe2vj',
 '005vBEORjw1ev1g9cjiylj30hq0vktan',
 '817561d7jw1e3yakgqkalj20at097ab1',
 '730c081ejw1eg8ho208xdj206s06r3ym',
 '4fca5397gw1dtjjwget52j',
 '640a6385tw1e607cximfxj20ac08ewfd',
 '006c4WZjjw1ev12ajgzs7j30ch0m8abz',
 '45206f21jw1dx8t75ppqwj',
 '5d6648f2jw1eeepc8d7yvj20hs0np77f',
 '6d431500jw1e3sa6m903cj20dc0m8q5n',
 '7c27db9fjw1dwsyufp10qj',
 '84f41de9jw1e2rtdl9g1cj',
 '5db6dc32gw1dzx0dm8pqxj',
 '498379fdjw1e8hcjpk4pjj20qe0ffacc',
 '7608d41djw1edmh42il34j20bc0au0t4',
 '5a359721jw1e2mxv1k6sqj',
 '670ea1ecjw1e5mfir8o7zj20c80fn3zn',
 'bd0a94d5jw1e15t0nybmij',
 '8d5ccf50jw1duu9h0j39uj',
 '005tWjn3jw1enqc6rijo7j30hs0co404',
 '6ef8a966gw1dwdkvpiv81j',
 '6b4378a6jw1dydfnwi130j',
 'b3c7dfc7jw1ev22zmmpy2j20ew0gq0v8',
 '690d0d1djw1er6qjorfvlj20im0hd0w7',
 'c5796fddjw1euaft47ej6j20hs0nzq3p',
 '644f8642jw1eez769j4bxj20hs0v

In [61]:
# generate from text
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        #sent_emb = sent_emb.repeat(batch_size,1)
        sent_emb = sent_emb.repeat(1,1)
        fake_imgs = netG(noise,sent_emb,eval=True).float()
        #name = f'{captions[i].replace(" ", "-")}'
        name=file_name[i]
        vutils.save_image(fake_imgs.data, './samples/%s.jpg'%(name), nrow=8, value_range=(-1, 1), normalize=True)

In [40]:
#image = os.path.join('/home/jovyan/work/GALIP/rumor_images','7fcc7edetw1e6btq03senj20c80gawgs.jpg')

In [42]:
#shutil.copy(image, "./real")

'./real/7fcc7edetw1e6btq03senj20c80gawgs.jpg'

In [38]:
#file_name[0]

'7fcc7edetw1e6btq03senj20c80gawgs'

In [64]:
for i in file_name:
    image = os.path.join('/home/jovyan/work/GALIP/rumor_images',f'{i}.jpg')
    a = os.listdir("/home/jovyan/work/GALIP/rumor_images/")
    if image in a:
        shutil.copy(image, "./real")
    else:
        image = os.path.join('/home/jovyan/work/GALIP/rumor_images',f'{i}.png')
        a = os.listdir("/home/jovyan/work/GALIP/rumor_images/")
        if image in a:
            shutil.copy(image, "./real")

In [67]:
a = os.listdir("./real")
a

['5db6dc32gw1dzx0dm8pqxj.jpg',
 '730c081ejw1eg8ho208xdj206s06r3ym.jpg',
 '96449d00jw1dyupdyukbkj.jpg',
 '8d5c03a7jw1e6r4cc7x02j20by0c40tt.jpg',
 '5a359721jw1e2mxv1k6sqj.jpg',
 '7fcc7edetw1e6btq03senj20c80gawgs.jpg',
 '3dc2ac30jw1dx457fr1owj.jpg',
 '4fca5397gw1dtjjwget52j.jpg',
 '7c27db9fjw1dwsyufp10qj.jpg',
 '.ipynb_checkpoints',
 '670ea1ecjw1e5mfir8o7zj20c80fn3zn.jpg',
 '6d431500jw1e3sa6m903cj20dc0m8q5n.jpg',
 '005vBEORjw1ev1g9cjiylj30hq0vktan.jpg',
 '498379fdjw1e8hcjpk4pjj20qe0ffacc.jpg',
 '7608d41djw1edmh42il34j20bc0au0t4.jpg',
 '84f41de9jw1e2rtdl9g1cj.jpg',
 '640a6385tw1e607cximfxj20ac08ewfd.jpg',
 '5d6648f2jw1eeepc8d7yvj20hs0np77f.jpg',
 '45206f21jw1dx8t75ppqwj.jpg',
 '5103ab97gw1dwmttofzfoj.jpg',
 '3d5b0116jw1e3s876pe2vj.jpg',
 '006c4WZjjw1ev12ajgzs7j30ch0m8abz.jpg',
 '817561d7jw1e3yakgqkalj20at097ab1.jpg']

In [71]:
count=0
for i in b:
    if i in a:
        count+=1

In [72]:
count

23

In [73]:
b = os.listdir("./samples")
for i in range(len(b)):
    if b[i] not in a:
        os.remove(os.path.join("./samples",b[i]))

In [75]:
b = os.listdir("./samples")
len(b)

23

In [77]:
pip install pytorch_fid

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [83]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorch_fid import fid_score

# 准备真实数据分布和生成模型的图像数据
real_images_folder = './real'
generated_images_folder = './samples'

# # 加载预训练的Inception-v3模型
# inception_model = torchvision.models.inception_v3(pretrained=True)

# # 定义图像变换
# transform = transforms.Compose([
#     transforms.Resize(299),
#     transforms.CenterCrop(299),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])

# 计算FID距离值
fid_value = fid_score.calculate_fid_given_paths([real_images_folder, 
                                                 generated_images_folder],batch_size=1, 
                                                device='cuda', dims=2048, num_workers=0)
print('FID value:', fid_value)

100%|██████████| 22/22 [00:01<00:00, 16.99it/s]


FID value: 405.56458954396635


In [82]:
fid_value

405.56458954396635